In [94]:
import numpy as np
import pandas as pd
import itertools
from collections import defaultdict

## Global parameters

In [137]:
binary_relevance = [0,1]
cut_off = 3

## Simulated rankings for system E and P

In [ ]:
system_e = list(map(list, itertools.product(binary_relevance, repeat=cut_off)))
system_p = list(map(list, itertools.product(binary_relevance, repeat=6)))

system_e

ids_e = [1,2,3]
ids_p = [1,2,3,4,5,6]

system_e = [list(zip(ranking, ids_e)) for ranking in system_e]
system_p = list(itertools.permutations([list(zip(ranking, ids_p)) for ranking in system_p], 6))
system_p
# ranking_pairs = list(itertools.product(system_e, system_p))
# ranking_pairs

## Defining Expected Reciprocal Rank @ Cut-off (ERR@-)

In [97]:
mapping_relevance_to_probability = lambda pos_g, max_g: ((2**pos_g) - 1 ) / (2**max_g)

In [98]:
def ERR(ranking, mapping=mapping_relevance_to_probability, n=cut_off):
    p, err = 1, 0
    
    for r in range(0, n):
        R = mapping_relevance_to_probability(ranking[r], 1)
        err += (p * (R/(r+1)))
        p *= (1-R)
        
    return err

## Construct the table

In [142]:
table = defaultdict(lambda:list())
for pair in ranking_pairs:
    
    err_e = ERR(pair[0])
    err_p = ERR(pair[1])
    delta_err = err_e - err_p
    if delta_err >= 0: # only consider cases where system e outperforms system p
        table[delta_err].append(pair)
    
table
# table = {k:sum(v) for k,v in table.items()}
    
# table = pd.DataFrame(table, index=[0]).T
# table.reset_index(level=0, inplace=True)

ValueError: Shape of passed values is (23, 8), indices imply (23, 1)

In [135]:
bins = np.linspace(0.05, 0.95 , 10)
groups = table.groupby(pandas.cut(table['index'], bins))
# groups.count()[0]

index
(0.05, 0.15]    54
(0.15, 0.25]    36
(0.25, 0.35]    31
(0.35, 0.45]    14
(0.45, 0.55]     9
(0.55, 0.65]     3
(0.65, 0.75]     7
(0.75, 0.85]     8
(0.85, 0.95]     1
Name: 0, dtype: int64

In [ ]:
def team_draft_interleaving(ranking_a, ranking_b):
    I, team_a, team_b = list(), set(), set()
    